In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests
import lyricsgenius

In [ ]:
def create_dataset(genres, count=5):
    k = int(count / 100)
    l = count % 100
    for i in range(k + 1):
        lyrics = []
        genre_names = []
        if i == k:
            count = l
            if count == 0:
                break
        else:
            count = 100
        for genre in genres:
            try:
                df = pd.DataFrame(sp.recommendations(seed_genres=[genre], limit=count, country=['GB', 'US'], min_popularity=70)['tracks'])[['artists', 'name']]
                df.artists = df.artists.apply(lambda x: x[0]['name'])
                for index, song in df.iterrows():
                    song_lyrics = genius.search_song(song['name'], song['artists'])
                    if song_lyrics is None:
                        continue
                    song_lyrics = song_lyrics.to_text()
                    lyrics.append(song_lyrics[song_lyrics.find('Lyrics') + 6:])
                    genre_names.append(genre)
            except:
                continue
        df = pd.DataFrame({
            'lyrics': lyrics,
            'genre': genre_names
        })
        df = df.sample(frac=1).reset_index(drop=True)
        df.to_csv('data/raw/dataset.csv', index=False, mode='a', header=False)

In [ ]:
cid_spotify = '39e61d63675943a6b81426b7507ee987'
secret_spotify = 'e8d971a5b9ff498692aaf12be038b994'
cat_genius = 'JgzUwU50wt6LFRGQK1wjreUsOQtKfwFCi_RP3Qbuk-vJyCTq1V4NY8GDDi_qXznl'

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid_spotify, client_secret=secret_spotify)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager, requests_timeout=10, retries=10)

genius = lyricsgenius.Genius(cat_genius, timeout=10, retries=10)

In [ ]:
print(list(sp.recommendation_genre_seeds()['genres']))

In [ ]:
create_dataset(['alternative', 'hip-hop', 'pop', 'rock'], 200)